# Trend & Anomaly Detection

This notebook demonstrates:
- Trend extraction and analysis
- Seasonality detection
- Anomaly detection methods
- Visualizing patterns and outliers


In [ ]:
# Import tools and libraries
import sys
from pathlib import Path
sys.path.insert(0, str(Path().absolute().parent))

from src.data_loader import load_cpg_data
from src.tools.trend_analysis import extract_trends, detect_seasonality, compare_stores
from src.tools.anomaly_detection import detect_anomalies
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8')
print("✅ Libraries imported successfully!")


In [ ]:
# Load data
data_path = '../data/cpg_sales_data.parquet'

try:
    df = load_cpg_data(data_path)
    print(f"✅ Data loaded: {df.shape[0]:,} rows")
except FileNotFoundError:
    print(f"❌ Data file not found at {data_path}")
    df = None


In [ ]:
# Detect seasonality
if df is not None:
    print("🔄 Detecting seasonality...")
    seasonality = detect_seasonality(
        df,
        date_col='date',
        value_col='revenue',
        period='weekly'
    )
    
    print(f"✅ Seasonality analysis complete!")
    print(f"\n📊 Seasonality Summary:")
    print(f"   Period: {seasonality.get('period', 'N/A')}")
    print(f"   Seasonality detected: {seasonality.get('has_seasonality', 'N/A')}")
    print(f"   Strength: {seasonality.get('seasonality_strength', 'N/A')}")
    
    # Visualize seasonality
    if 'seasonal_pattern' in seasonality:
        fig, ax = plt.subplots(figsize=(12, 6))
        seasonal_df = seasonality['seasonal_pattern']
        ax.plot(seasonal_df.index, seasonal_df.values, marker='o')
        ax.set_title('Seasonal Pattern')
        ax.set_xlabel('Period')
        ax.set_ylabel('Average Revenue')
        plt.tight_layout()
        plt.show()


## Anomaly Detection


In [ ]:
# Detect anomalies
if df is not None:
    print("🔍 Detecting anomalies...")
    anomalies = detect_anomalies(
        df,
        date_col='date',
        value_col='revenue',
        method='zscore',
        threshold=3.0
    )
    
    print(f"✅ Anomaly detection complete!")
    print(f"\n📊 Anomaly Summary:")
    print(f"   Total anomalies found: {len(anomalies.get('anomalies', []))}")
    print(f"   Anomaly rate: {anomalies.get('anomaly_rate', 0):.2%}")
    
    # Visualize anomalies
    if 'anomalies' in anomalies and len(anomalies['anomalies']) > 0:
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # Plot all data
        daily_revenue = df.groupby('date')['revenue'].sum()
        ax.plot(daily_revenue.index, daily_revenue.values, label='Normal', alpha=0.7)
        
        # Highlight anomalies
        anomaly_dates = [a['date'] for a in anomalies['anomalies']]
        anomaly_values = [a['value'] for a in anomalies['anomalies']]
        ax.scatter(anomaly_dates, anomaly_values, color='red', s=100, 
                  label='Anomalies', zorder=5)
        
        ax.set_title('Anomaly Detection Results')
        ax.set_xlabel('Date')
        ax.set_ylabel('Revenue')
        ax.legend()
        ax.tick_params(axis='x', rotation=45)
        plt.tight_layout()
        plt.show()
        
        # Show top anomalies
        print("\n🚨 Top 5 Anomalies:")
        for i, anomaly in enumerate(anomalies['anomalies'][:5], 1):
            print(f"   {i}. Date: {anomaly['date']}, Value: {anomaly['value']:.2f}, "
                  f"Z-score: {anomaly.get('z_score', 'N/A'):.2f}")
